In [2]:

import pandas as pd
from bcb import sgs, Expectativas
from datetime import datetime, timedelta
import numpy as np

In [3]:
# Endpoints
em = Expectativas()
# em.describe()
ep_m_mensal = em.get_endpoint('ExpectativaMercadoMensais')

In [4]:
# função para fazer a formatação das DataFrames
def formata_df(df: pd.DataFrame):
    
    # Data and DataRef to datetime
    df['DataReferencia'] = pd.to_datetime(df['DataReferencia'], format='%m/%Y')
    
    df['Data'] = pd.to_datetime(df['Data'], format='%Y-%m-%d')
    df['Data']= df['Data'].dt.strftime('%d/%m/%Y')
    
    # table pivot - table sorting by header - formating header
    df_pivot = df.pivot_table(index='Data', columns='DataReferencia', values='Mediana')
    df_pivot = df_pivot.sort_index(axis=1)
    df_pivot.columns = df_pivot.columns.strftime('%m/%Y')

    # Formating Data
    df_pivot = df_pivot.sort_index(key=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))

    # Formatando casas decimais e alterando dtypes=float
    df_pivot = df_pivot.astype(float)
    
    return df_pivot



In [12]:
# IPCA mensal ultimos 30 dias
ipca_mensal = ep_m_mensal.query().filter(ep_m_mensal.Indicador == 'IPCA', ep_m_mensal.Data >= '2024-01-01', ep_m_mensal.baseCalculo == '0').select(ep_m_mensal.Indicador, ep_m_mensal.Data, ep_m_mensal.Mediana, ep_m_mensal.DataReferencia).collect()
ipca_mensal_pivot = formata_df(ipca_mensal)




In [6]:
# IPCA Livre 
ipca_livre_expec = ep_m_mensal.query().filter(ep_m_mensal.Indicador == 'IPCA Livres', ep_m_mensal.Data >= '2024-01-01', ep_m_mensal.baseCalculo == '0').select(ep_m_mensal.Indicador, ep_m_mensal.Data, ep_m_mensal.Mediana, ep_m_mensal.DataReferencia).collect()
ipca_livre_pivot = formata_df(ipca_livre_expec)


In [7]:
# IPCA Adm
ipca_adm_expec = ep_m_mensal.query().filter(ep_m_mensal.Indicador == 'IPCA Administrados', ep_m_mensal.Data >= '2024-01-01', ep_m_mensal.baseCalculo == '0').select(ep_m_mensal.Indicador, ep_m_mensal.Data, ep_m_mensal.Mediana, ep_m_mensal.DataReferencia).collect()
ipca_adm_pivot = formata_df(ipca_adm_expec)

In [8]:
# to excel
with pd.ExcelWriter(r'C:\Users\rm170600\Documents\Python\Projeto_API_SGS\Expec_IPCA.xlsx') as writer:
   ipca_mensal_pivot.to_excel(writer, sheet_name='IPCA Mensal')
   ipca_livre_pivot.to_excel(writer, sheet_name='IPCA Livre')
   ipca_adm_pivot.to_excel(writer, sheet_name='IPCA ADM')
